In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
nltk.download('punkt')
import tflearn
import random
import tensorflow
import json
with open('clg_data.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to C:\Users\Harika
[nltk_data]     Kasimahanthi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\Harika Kasimahanthi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Harika Kasimahanthi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Harika Kasimahanthi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, i

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
words = []
labels = []
docs_x = []
docs_y = []

In [3]:
for intent in data['intents']:
    for pattern in intent['Query']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["Tags"])
        
    if intent['Tags'] not in labels:
        labels.append(intent['Tags'])

In [4]:

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

In [5]:

tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.03174 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.03174 - acc: 0.9999 -- iter: 32/34
Training Step: 5000  | total loss: 0.03047 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.03047 - acc: 0.9999 -- iter: 34/34
--
INFO:tensorflow:D:\BTECH\Projects\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        print(bag_of_words(inp, words))
        print(results)
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['Tags'] == tag:
                responses = tg['Response']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[1.6212782e-08 1.6662108e-06 3.4369643e-07 5.2313106e-03 2.1475430e-03
  5.5853216e-06 9.7945422e-01 5.6043379e-03 7.5440579e-03 1.0898734e-05
  7.1224482e-10 8.5773992e-09 5.1346547e-08]]
nice to meet you
You: how are you
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[[1.2005722e-09 1.9886891e-06 8.7397331e-03 9.6367246e-01 1.8427710e-06
  1.1464553e-09 8.5351774e-03 1.8141489e-02 5.7455409e-06 8.8780833e-04
  1.5182157e-07 1.3659297e-05 2.8369101e-08]]
fantastic thank you!
